# Container - Working with ContentInstances


In this notebook you will learn how to store and handle data by using \<container> and \<contentInstance> resources.

In this notebook we will continue to use the resource structure created in the previous notebooks of this lecture.


- Store and retrieve data by using a \<contentInstance> resource
- Update the data under a \<container> resource
- Retrieve the latest and oldest \<contentInstance> resources


## Intitialization

<div class="alert alert-block alert-info" style="border-radius: 10px;">
The following section imports and initializes necessary functions and configurations. 
It must be executed once for each notebook before running any of cells.</div>

In [ ]:
%run ../src/init.py

## Store data using a \<contentInstance> resource

In a previous notebook Alice prepared the resource structure by registering an \<AE> and creating a \<container> child resource.

Let's now add a very simple \<contentInstance> resource to store an important message.

In [ ]:
CREATE (                                                      # CREATE request
    target                  = f'{cseRN}/aeAlice/container',   # Create the <cin> under aeAlice's <cnt>
    originator              = 'CAlice',                       # Request's originator (mandatory)
    requestIdentifier       = '123',                          # Request identifier (mandatory)
    releaseVersionIndicator = '3',                            # Request's release version indicator (mandatory)
    resourceType            = Type.ContentInstance,           # The request creates a <cin> resource
    
    # Request Content
    content = {
        "m2m:cin": {
            "rn" : "one",                                     # Assign a nice resource name so that we can access the resource easier
            "con": "hello, world"                             # Only add some content
        }
    }
)

As you can see from the resource tree the \<container> resource has now three child resources: the new \<contentInstance> resource, and the two virtual resources \<oldest> and \<latest>.

<div class="alert alert-block alert-success" style="border-radius: 10px">
    In the last request we assigned a <i>resourceName</i> to the resource so that we can easily retrieve it using its name in the following request. But in most cases it would not be feasible for an AE to assign a different <i>resourceName</i> for each ContentInstance resource and also to keep track of it. Therefore, in future CREATE requests in these exercises we will mostly leave it out, too.
</div>

## Retrieve the \<contentInstance>

Alice will retrieve the just added \<contentInstance> resource from the CSE with the following request.

In [ ]:
RETRIEVE (                                                       # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/container/one',  # Target resource is the <cin> under Alice's <cnt>
    originator              = 'CAlice',                          # Request's originator is CAlice (mandatory)
    requestIdentifier       = '123',                             # Request identifier (mandatory)
    releaseVersionIndicator = '3'                                # Request's release version indicator (mandatory)
)

## Update the \<contentInstance>

Now Alice will try to update the \<contentInstance resource with an UPDATE request.

In [ ]:
UPDATE (                                                        # UPDATE request
    target                  = f'{cseRN}/aeAlice/container/one', # Update the <cin> under aeAlice's <cnt>
    originator              = 'CAlice',                         # Request's originator (mandatory)
    requestIdentifier       = '123',                            # Request identifier (mandatory)
    releaseVersionIndicator = '3',                              # Request's release version indicator (mandatory)
    
    # Request Content
    content = {
        "m2m:cin": {
            "con": "hello, CSE"                                 # Update the content
        }
    }
)

This request **fails**, because the \<contentInstance> resource cannot be updated. 

<div class="alert alert-block alert-success" style="border-radius: 10px">
You should treat it as one version of the data that it contains. All &lt;contentInstance> resources under a &lt;container> are usually different versions of the same data point.
</div>

## Add a new \<contentInstance> resource

Instead of updating the existing \<contentInstance> resource Alice will now add a new resource with the changed content data. And while she's at it she will fill in the *contentInfo* attribute as well.

<div class="alert alert-block alert-success" style="border-radius: 10px">
    The <i>contentInfo</i> attribute is a list of colon-separated values that provide data consumers with additional information how to decode and process the data in the <i>content</i> attribute:  
    
<li>what it is: MIME type (this part is mandatory)</li>
<li>how it is encoded: either not encoded, base64 on string data, or base64 on binary data (this part is mandatory as well)</li>
<li>what content security has been applied (this is optional, the default is none).
    The CSE itself does not process the values in the <i>contentInfo</i> or the <i>content</i> attributes.
</div>


In [ ]:
CREATE (                                                      # CREATE request
    target                  = f'{cseRN}/aeAlice/container',   # Create the <cin> under aeAlice's <cnt>
    originator              = 'CAlice',                       # Request's originator (mandatory)
    requestIdentifier       = '123',                          # Request identifier (mandatory)
    releaseVersionIndicator = '3',                            # Request's release version indicator (mandatory)
    resourceType            = Type.ContentInstance,           # The request creates a <cin> resource
    
    # Request Content
    content = {
        "m2m:cin": {
            "rn" : "two",                                     # Assign another nice resource name so that we can access the resource easier
            "cnf": "application/text:0",                      # Content type & encoding (0=none)
            "con": "hello, CSE"                               # "update" the content
        }
    }
)

This request created another \<contentInstance>. As you can see from the response and the tree above a new resource was added to the \<container> that represents the new value. You can also see that the *contentInfo* (*cnf*) attribute is set.

# Retrieve the \<latest> and \<oldest> resources

There could be a large number of \<contentInstance> resources under a \<container> resource. It would be very helpful for Alice if she could retrieve the latest or the oldest of these resource.

With the following two request we can retrieve these resources.

<div class="alert alert-block alert-success" style="border-radius: 10px">
&lt;latest> and &lt;oldest> are virtual resources that automatically point to the latest and oldest &lt;contentInstance> resources under a &lt;container> resource, respectively. Sending a RETRIEVE request to one of these resources will return the corresponding &lt;contentInstance> resource.
</div>

In [ ]:
RETRIEVE (                                                       # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/container/la',   # Target resource is the <latest> virtual resource
    originator              = 'CAlice',                          # Request's originator is CAlice (mandatory)
    requestIdentifier       = '123',                             # Request identifier (mandatory)
    releaseVersionIndicator = '3'                                # Request's release version indicator (mandatory)
)

The \<contentInstance> resource "two" was the last one created by Alice, so this resource is returned in this request

In [ ]:
RETRIEVE (                                                       # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/container/ol',   # Target resource is the <oldest> virtual resource
    originator              = 'CAlice',                          # Request's originator is CAlice (mandatory)
    requestIdentifier       = '123',                             # Request identifier (mandatory)
    releaseVersionIndicator = '3'                                # Request's release version indicator (mandatory)
)

This time the request returns the \<contentInstance> resource "one" (which Alice created first).

<div class="alert alert-block alert-success" style="border-radius: 10px">
 If you have a closer look at the retrieved resources you will notice the attribute <i>st</i> (or <i>state</i>). This attribute contains a kind of sequence number, applied by the parent &lt;container> resource. This number is used internally by the CSE but can also be used by an AE to process  &lt;contentInstance> resources the right order.    
</div>